In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr
%config InlineBackend.figure_format = 'retina'

Columns:
- 1-14 Source Time (GMT) YYYYDDDHHMMSSS
- 16-17 Number of recording hydrophones
- 18-32 ID's of recording hydrophones (in order received)
- 34-41 Epicenter Latitude (Degrees, N=+, S=-)
- 42-50 Epicenter Longitude (Degrees, E=+, W=-)
- 51-57 Estimated error in latitude (degrees)
- 58-64 Estimated error in longitude (degrees)
- 65-71 Estimated error in source time (seconds)
- 72-79 Source Magnitude (dB re 1 micro-Pa @ 1m)

## Form PANDAS dataframe

In [2]:
col_names = ['source_time', 'h_count', 'h_id', 'ep_lat', 'ep_lon', 'er_lat', 'er_lon',
        'er_time', 'mdb', 'mdb2','event'] 

In [3]:
# concat all txt files 
#cat Data/* > merged_data.txt

In [4]:
df_pandas = pd.read_csv('/home/jackiesinger/Data/bravoseis_data/hydroacoustic_data/merged_data/merged_data.txt', sep='\s+', header=None, usecols=[0,1,2,3,4,5,6,7,8,9,10], names=col_names)
df_pandas.head()

,source_time,h_count,h_id,ep_lat,ep_lon,er_lat,er_lon,er_time,mdb,mdb2,event
0,20190140137360,3,231,-62.837,-59.316,0.063,0.083,2.410,193.15,193.4,IQ
1,20190140151534,5,23456,-65.639,-65.407,1.194,2.462,4.236,222.31,222.7,IQ
2,20190140226551,6,654321,-62.201,-57.105,0.063,0.085,4.642,203.04,203.4,IQ
3,20190140332141,4,3241,-62.379,-59.106,0.024,0.020,1.564,195.50,195.7,IDK
4,20190140352221,6,654321,-62.194,-57.103,0.068,0.094,5.158,199.77,200.2,IDK


In [5]:
df_pandas['time'] = pd.to_datetime(df_pandas['source_time'], format='%Y%j%H%M%S%f')
df_pandas.drop('source_time', axis=1, inplace=True)

df_pandas.head()

,h_count,h_id,ep_lat,ep_lon,er_lat,er_lon,er_time,mdb,mdb2,event,time
0,3,231,-62.837,-59.316,0.063,0.083,2.410,193.15,193.4,IQ,2019-01-14 01:37:36.000
1,5,23456,-65.639,-65.407,1.194,2.462,4.236,222.31,222.7,IQ,2019-01-14 01:51:53.400
2,6,654321,-62.201,-57.105,0.063,0.085,4.642,203.04,203.4,IQ,2019-01-14 02:26:55.100
3,4,3241,-62.379,-59.106,0.024,0.020,1.564,195.50,195.7,IDK,2019-01-14 03:32:14.100
4,6,654321,-62.194,-57.103,0.068,0.094,5.158,199.77,200.2,IDK,2019-01-14 03:52:22.100


In [6]:
df_pandas.describe()

,h_count,h_id,ep_lat,ep_lon,er_lat,er_lon,er_time,mdb,mdb2
count,17575.000000,17575.000000,17575.000000,17575.000000,17575.000000,17575.00000,17575.000000,17575.000000,17575.000000
mean,4.461451,120409.779459,-62.599793,-58.126725,0.179326,0.36262,10.811506,196.583986,196.840319
std,1.186722,202710.779401,0.912457,2.177078,0.809635,1.56670,20.138662,8.252425,8.295866
min,3.000000,123.000000,-81.352000,-80.049000,0.000000,0.00000,0.000000,169.410000,169.600000
25%,3.000000,654.000000,-62.904000,-59.206000,0.025000,0.03100,1.224000,190.780000,191.000000
50%,4.000000,6543.000000,-62.616000,-58.176000,0.052000,0.05600,3.104000,196.410000,196.600000
75%,6.000000,123456.000000,-62.315000,-57.199500,0.135000,0.19900,9.516000,201.960000,202.200000
max,6.000000,654321.000000,-45.465000,-23.135000,68.835000,61.27300,99.999000,240.970000,241.400000


In [7]:
df_pandas.set_index(['ep_lat', 'ep_lon', 'time'])

,,,h_count,h_id,er_lat,er_lon,er_time,mdb,mdb2,event
ep_lat,ep_lon,time,,,,,,,,
-62.837,-59.316,2019-01-14 01:37:36.000,3,231,0.063,0.083,2.410,193.15,193.4,IQ
-65.639,-65.407,2019-01-14 01:51:53.400,5,23456,1.194,2.462,4.236,222.31,222.7,IQ
-62.201,-57.105,2019-01-14 02:26:55.100,6,654321,0.063,0.085,4.642,203.04,203.4,IQ
-62.379,-59.106,2019-01-14 03:32:14.100,4,3241,0.024,0.020,1.564,195.50,195.7,IDK
-62.194,-57.103,2019-01-14 03:52:22.100,6,654321,0.068,0.094,5.158,199.77,200.2,IDK
...,...,...,...,...,...,...,...,...,...,...
-63.246,-58.976,2020-02-10 22:44:27.600,4,2134,0.068,0.053,4.599,206.79,207.1,IDK
-62.344,-56.975,2020-02-10 23:25:06.800,5,65432,0.034,0.176,5.468,197.71,198.1,IQ
-64.003,-61.487,2020-02-10 23:26:32.100,3,123,0.449,0.701,41.170,201.40,201.6,IDK


In [7]:
# df_multiindex = pd.DataFrame({'lat': df_pandas.ep_lat,
                              #'lon': df_pandas.ep_lon,
                              #'time': df_pandas.time,
                              #'h_count': df_pandas.h_count,
                              #'h_id': df_pandas.h_id,
                              #'er_lat': df_pandas.er_lat,
                              #'er_lon': df_pandas.er_lon,
                              #'er_time': df_pandas.er_time,
                              #'mdb': df_pandas.mdb,
                              #'mdb2': df_pandas.mdb2,
                              #'event': df_pandas.event})

In [8]:
#df_multiindex = df_multiindex.set_index(['lat', 'lon', 'time'])

In [ ]:
#df_multiindex

## Convert to XArray

In [ ]:
#crash occurs on this line
#df_multiindex.to_xarray()


In [8]:
df_xr = df_pandas.to_xarray()

In [9]:
df_xr.head()

<xarray.Dataset>
Dimensions:  (index: 5)
Coordinates:
  * index    (index) int64 0 1 2 3 4
Data variables:
    h_count  (index) int64 3 5 6 4 6
    h_id     (index) int64 231 23456 654321 3241 654321
    ep_lat   (index) float64 -62.84 -65.64 -62.2 -62.38 -62.19
    ep_lon   (index) float64 -59.32 -65.41 -57.1 -59.11 -57.1
    er_lat   (index) float64 0.063 1.194 0.063 0.024 0.068
    er_lon   (index) float64 0.083 2.462 0.085 0.02 0.094
    er_time  (index) float64 2.41 4.236 4.642 1.564 5.158
    mdb      (index) float64 193.2 222.3 203.0 195.5 199.8
    mdb2     (index) float64 193.4 222.7 203.4 195.7 200.2
    event    (index) object 'IQ' 'IQ' 'IQ' 'IDK' 'IDK'
    time     (index) datetime64[ns] 2019-01-14T01:37:36 ... 2019-01-14T03:52:...

df_xr.coords['ep_lon'] 
df_xr.coords['ep_lat'] `
df_xr

In [14]:
df_xr.set_coords(['ep_lat', 'ep_lon', 'time'])

<xarray.Dataset>
Dimensions:  (index: 17575)
Coordinates:
  * index    (index) int64 0 1 2 3 4 5 6 ... 17569 17570 17571 17572 17573 17574
    ep_lat   (index) float64 -62.84 -65.64 -62.2 -62.38 ... -64.0 -62.07 -62.59
    ep_lon   (index) float64 -59.32 -65.41 -57.1 -59.11 ... -61.49 -56.92 -57.27
    time     (index) datetime64[ns] 2019-01-14T01:37:36 ... 2020-02-10T23:58:...
Data variables:
    h_count  (index) int64 3 5 6 4 6 5 5 3 6 4 5 3 3 ... 3 5 4 4 4 6 4 4 5 3 6 4
    h_id     (index) int64 231 23456 654321 3241 ... 65432 123 654321 4653
    er_lat   (index) float64 0.063 1.194 0.063 0.024 ... 0.034 0.449 0.281 0.043
    er_lon   (index) float64 0.083 2.462 0.085 0.02 ... 0.176 0.701 0.579 0.065
    er_time  (index) float64 2.41 4.236 4.642 1.564 ... 5.468 41.17 28.73 2.992
    mdb      (index) float64 193.2 222.3 203.0 195.5 ... 197.7 201.4 199.3 198.6
    mdb2     (index) float64 193.4 222.7 203.4 195.7 ... 198.1 201.6 199.7 198.9
    event    (index) object 'IQ' 'IQ' 'IQ' 'IDK' 'IDK' ... 'IQ' 'IDK' 'IDK' 'IQ'

## Parse out Network

In [ ]:
df_network = df.loc[df['event'] == 'EQ']
cond1 = df_network["ep_lat"] < -62   
cond2 = df_network["ep_lat"] > -63.5
cond3 = df_network["ep_lon"] > -61
cond4 = df_network["ep_lon"] < -56.5

df_network = df_network[cond1 & cond2 & cond3 & cond4]
df_network.describe()

In [ ]:
df = df.groupby('event')
IQ = df.get_group('IQ')
EQ = df.get_group('EQ')
IDK = df.get_group('IDK')

In [ ]:
fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot()

ax.scatter(IQ.ep_lon, IQ.ep_lat, alpha=0.1, c='dimgrey', label='Ice Quake')
ax.scatter(EQ.ep_lon, EQ.ep_lat, c='cornflowerblue', alpha=0.1, label='Earthquake')
ax.grid()
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Events')
ax.legend();

In [ ]:
fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot()

ax.scatter(df_network.ep_lon, df_network.ep_lat, alpha=0.1, c='dimgrey')
ax.grid()
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Events');